## The P0 Compiler
#### COMP SCI 4TB3/6TB3, McMaster University
#### Original Author: Emil Sekerinski, revised March 2020

This collection of _Jupyter notebooks_ develops a compiler for P0, a programming langauge inspired by Pascal, as Pascal was designed for ease of compilation. The compiler currently generates WebAssebmbly and MIPS code, but is modularized to facilitate other targets. WebAssembly is representive of stack-based virtual machines while the MIPS architecture is representative of Reduced Instruction Set Computing (RISC) processors.

### The P0 Language
The main syntactic elements of P0 are *statements*, *declarations*, *types*, and *expressions*.

#### Statements
* _Assignment statement_ (`d` designator, `e` expressions):
      d := e
* _Procedure call_ (`p` procedure identifier, `e₁`, `e₂`, … expressions, `d` designator):
      p(e₁, e₂, …)
      d ← p(e₁, e₂, …)
* _Sequential composition_ (`S₁`, `S₂`, … statements):
      S₁; S₂; …
* _If-statements_ (`B` Boolean expression, `S`, `T` statements):
	  if B then S
      if B then S else T
* _While-statements_ (`B` Boolean expression, `S` statement):
      while B do S

#### Declarations
* _Constant declaration_ (`c` constant identifier, `e` constant expression):
      const c =  e
* _Type declaration_ (`t` type identifier, `T` type):
      type t = T
* _Variable declaration_ (`x₁`, `x₂`, … variable identifiers, `T` type):
      var x₁, x₂, …: T
* _Procedure declaration_ (`p` procedure identifier, `v₁`, `v₂`, … variable identifiers, `T₁`, `T₂`, …, `U`  types, `D₁`, `D₂`, … declarations, `S` statement):
      procedure p (v₁: T₁, v₂: T₂, …) → (r: U)
          D₁
          D₂
          …
              S

#### Types
* _Elementary Types:_
      integer, boolean
* _Arrays_ (`m`, `n` integer expressions, `T` type):
      m .. n → T

#### Expressions:
* _Constants:_
	  number, identifier
* _Designator_ (`x` identifier, `i` expression):
      x
	  x[i]
* _Operators,_ in order of their binding power (e, e₁, e₂ are expressions):
	  (e), ¬ e
      e₁ × e₂, e₁ div e₂, e₁ mod e₂, e₁ and e₂
      + e, – e, e₁ + e₂, e₁ – e₂, e₁ or e₂
      e₁ = e₂, e₁ ≠ e₂, e₁ < e₂, e₁ ≤ e₂, e₁ > e₂, e₁ ≥ e₂

Types `integer` and `boolean` and procedures `read`, `write`, `writeln` are not symbols of the grammar; they are _standard identifiers_ (*predefined identifiers*).

### P0 Examples


```Pascal
procedure quot(x, y: integer) → (q: integer)
    var r: integer
      q := 0; r := x
      while r ≥ y do { q × y + r = x ∧ r ≥ y }
        r := r - y; q := q + 1

program arithmetic
    var x, y, q, r: integer
      x ← read(); y ← read()
      q ← quot(x, y)
      write(q); writeln()
```

```Pascal
procedure fact(n: integer) → (f: integer)
    if n = 0 then f := 1
    else
        f ← fact(n - 1); f := f × n

program factorial;
    var y, z: integer
        y ← read(); z ← fact(y); write(z)
```

```Pascal
const N = 10
var a: 0 .. N - 1 → integer

procedure has(x: integer) → (r: boolean)
    var i: integer
        i := 0
        while i < N and a[i] ≠ x do i := i + 1
        r := i < N

```

### The P0 Grammar

    designator ::= "malloc" ident ":" type | ident { "[" expression "]" }
    factor ::= designator | integer | "(" expression ")" | "¬" factor
    term ::= factor {("×" | "div" | "mod" | "and") factor}
    simpleExpression ::= ["+" | "-"] term {("+" | "-" | "or") term}
    expression ::= simpleExpression
        {("=" | "≠" | "<" | "≤" | ">" | "≥") simpleExpression}
    statementList ::= statement {";" statement}
    statementBlock ::= statementList {statementList}
    statementSuite ::= statementList | INDENT statementBlock DEDENT
    statement ::=
        designator ":=" expression |
        designator "←" ident "(" [expression {"," expression}] ")" |
        "if" expression "then" statementSuite ["else" statementSuite] |
        "while" expression "do" statementSuite
    type ::=
        ident |
        expression ".." expression "→" type
    typedIds ::= ident ":" type {"," ident ":" type}
    declarations ::= 
        {"const" ident "=" expression}
        {"type" ident "=" type}
        {"var" typedIds}
        {"procedure" ident "(" [typedIds] ")" [ "→" "(" typedIds ")" ] body}
    body ::= INDENT declarations (statementBlock | INDENT statementBlock DEDENT) DEDENT
    program ::= declarations "program" ident body

### Modularization
<div><span style="float:right"><img width="60%" src="modularization.svg"/></span></div>

- The parser, `P0`, parses the source text, type-checks it, evaluates constant expressions, and generates target code, in one pass over the source text.
- The scanner, `SC`, reads characters of the source text and provides the next symbol to the parser; it allows errors to be reported at the current position in the source text.
- The symbol table, `ST`, stores all currently valid declarations, as needed for type-checking.
- The code generator, `CG`, provides the parser with procedures for generating code for P0 expressions, statements, and variable declarations, and procedure declarations.

The parser is the main program that calls the scanner, symbol table, and code generator. All call the scanner for error reporting. The code generator augments the entries in the the symbol table, for example with the size and location of variables. There are three code generators: `CCGwat` generates WebAssembly code, `CGmips` generates MIPS code, and `CGast` generates only an abstract syntax tree.

### The Parser
The scanner and symbol table are always imported. Depending on the selected target, a different code generator is imported when compilation starts.

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
import SC  #  used for SC.init, SC.sym, SC.val, SC.error
from SC import TIMES, DIV, MOD, AND, PLUS, MINUS, OR, EQ, NE, LT, GT, \
    LE, GE, PERIOD, COMMA, COLON, NOT, LPAREN, RPAREN, LBRAK, RBRAK, \
    LARROW, RARROW, OF, THEN, DO, BECOMES, NUMBER, IDENT, SEMICOLON, ELSE, \
    IF, WHILE, ARRAY, RECORD, CONST , TYPE, VAR, PROCEDURE, PROGRAM, \
    INDENT, DEDENT, NL, EOF, getSym, mark, MALLOC

import ST  #  used for ST.init
from ST import Var, Ref, Const, Type, Proc, StdProc, Int, Bool, Enum, \
    Record, Array, newDecl, find, openScope, topScope, closeScope

The first sets for recursive descent parsing are:

In [1]:
FIRSTFACTOR = {IDENT, NUMBER, LPAREN, NOT}
FIRSTEXPRESSION = {PLUS, MINUS, IDENT, NUMBER, LPAREN, NOT}
FIRSTSTATEMENT = {MALLOC, IDENT, IF, WHILE}
FIRSTTYPE = {IDENT, RECORD, ARRAY, LPAREN}
FIRSTDECL = {CONST, TYPE, VAR, PROCEDURE}

NameError: name 'IDENT' is not defined

Procedure `designator()` parses

    designator ::= ident { "[" expression "]" }

and generates code for the designator if not error is reported. If the designator is a constant, a `Const` item is returned (and code may not be generated); if the designator is not a constant, the location of the result is returned. 

In [2]:
def designator():
    x = find(SC.val)
    if type(x) == Var: x = CG.genVar(x); getSym()
    elif type(x) == Const: x = Const(x.tp, x.val); x = CG.genConst(x); getSym()
    else: mark('designator expected')
    while SC.sym == LBRAK:
        getSym(); y = expression()
        if type(x.tp) == Array:
            if y.tp == Int:
                if type(y) == Const and (y.val < x.tp.lower or y.val >= x.tp.lower + x.tp.length):
                    mark('index out of bounds')
                else: x = CG.genIndex(x, y)
            else: mark('index not integer')
        else: mark('not an array')
        if SC.sym == RBRAK: getSym()
        else: mark("] expected")
    return x

Procedure `factor()` parses

    factor ::= designator | integer | "(" expression ")" | "¬" factor.

and generates code for the factor if no error is reported. If the factor is a constant, a `Const` item is returned (and code may not be generated); if the factor is not a constant, the location of the result is returned. 

In [ ]:
def factor():
    if SC.sym == IDENT: x = designator()
    elif SC.sym == NUMBER:
        x = Const(Int, SC.val); x = CG.genConst(x); getSym()
    elif SC.sym == LPAREN:
        getSym(); x = expression()
        if SC.sym == RPAREN: getSym()
        else: mark(") expected")
    elif SC.sym == NOT:
        getSym(); x = factor()
        if x.tp != Bool: mark('not boolean')
        elif type(x) == Const: x.val = 1 - x.val # constant folding
        else: x = CG.genUnaryOp(NOT, x)
    else: mark("expression expected")
    return x

Procedure `term()` parses

    term ::= factor {("×" | "div" | "mod" | "and") factor}.

and generates code for the term if no error is reported. If the term is a constant, a `Const` item is returned (and code may not be generated); if the term is not a constant, the location of the result is returned. 

In [ ]:
def term():
    x = factor()
    while SC.sym in {TIMES, DIV, MOD, AND}:
        op = SC.sym; getSym();
        if op == AND and type(x) != Const: x = CG.genUnaryOp(AND, x)
        y = factor() # x op y
        if x.tp == Int == y.tp and op in {TIMES, DIV, MOD}:
            if type(x) == Const == type(y): # constant folding
                if op == TIMES: x.val = x.val * y.val
                elif op == DIV: x.val = x.val // y.val
                elif op == MOD: x.val = x.val % y.val
            else: x = CG.genBinaryOp(op, x, y)
        elif x.tp == Bool == y.tp and op == AND:
            if type(x) == Const: # constant folding
                if x.val: x = y # if x is true, take y, else x
            else: x = CG.genBinaryOp(AND, x, y)
        else: mark('bad type')
    return x

Procedure `simpleExpression()` parses

    simpleExpression ::= ["+" | "-"] term {("+" | "-" | "or") term}.

and generates code for the simple expression if no error is reported. If the simple expression is a constant, a `Const` item is returned (and code may not be generated); if the simple expression is not constant, the location of the result is returned. 

In [ ]:
def simpleExpression():
    if SC.sym == PLUS:
        getSym(); x = term()
    elif SC.sym == MINUS:
        getSym(); x = term()
        if x.tp != Int: mark('bad type')
        elif type(x) == Const: x.val = - x.val # constant folding
        else: x = CG.genUnaryOp(MINUS, x)
    else: x = term()
    while SC.sym in {PLUS, MINUS, OR}:
        op = SC.sym; getSym()
        if op == OR and type(x) != Const: x = CG.genUnaryOp(OR, x)
        y = term() # x op y
        if x.tp == Int == y.tp and op in {PLUS, MINUS}:
            if type(x) == Const == type(y): # constant folding
                if op == PLUS: x.val = x.val + y.val
                elif op == MINUS: x.val = x.val - y.val
            else: x = CG.genBinaryOp(op, x, y)
        elif x.tp == Bool == y.tp and op == OR:
            if type(x) == Const: # constant folding
                if not x.val: x = y # if x is false, take y, else x
            else: x = CG.genBinaryOp(OR, x, y)
        else: mark('bad type')
    return x

Procedure `expression()` parses

    expression ::= simpleExpression
                 {("=" | "≠" | "<" | "≤" | ">" | "≥") simpleExpression}.

and generates code for the expression if no error is reported. If the expression is a constant, a `Const` item is returned (and code may not be generated); if the expression is not constant, the location of the result is returned. 

In [ ]:
def expression():
    x = simpleExpression()
    while SC.sym in {EQ, NE, LT, LE, GT, GE}:
        op = SC.sym; getSym(); y = simpleExpression() # x op y
        if x.tp == y.tp in (Int, Bool):
            if type(x) == Const == type(y): # constant folding
                if op == EQ: x.val = x.val == y.val
                elif op == NE: x.val = x.val != y.val
                elif op == LT: x.val = x.val < y.val
                elif op == LE: x.val = x.val <= y.val
                elif op == GT: x.val = x.val > y.val
                elif op == GE: x.val = x.val >= y.val
                x.tp = Bool
            else: x = CG.genRelation(op, x, y)
        else: mark('bad type')
    return x

Procedure `statementList()` parses

    statementList ::= statement {";" statement}

and generates code for the statement list if no error is reported.

In [ ]:
def statementList():
    x = statement()
    while SC.sym == SEMICOLON:
        getSym(); y = statement(); x = CG.genSeq(x, y)
    return x

Procedure `statementBlock()` parses

    statementBlock ::= statementList {statementList}

and generates code for the statement block if no error is reported. Each statement list has to start on a new line.

In [ ]:
def statementBlock():
    x = statementList()
    while SC.sym in FIRSTSTATEMENT:
        if not SC.newline: mark('new line expected')
        y = statementList(); x = CG.genSeq(x, y)
    return x

Procedure `statementSuite()` parses

    statementSuite ::= statementList | INDENT statementBlock DEDENT

and generates code for the statement suite if no error is reported.

In [ ]:
def statementSuite():
    if SC.sym in FIRSTSTATEMENT: x = statementList()
    elif SC.sym == INDENT:
        getSym(); x = statementBlock()
        if SC.sym == DEDENT: getSym();
        else: mark('dedent expected')
    else: mark("(indented) statement expected!")
    return x

Procedure `statement()` parses

    statement ::=
        designator ":=" expression |
        designator "←" ident "(" [expression {"," expression}] ")" |
        "if" expression "then" statementSuite ["else" statementSuite] |
        "while" expression "do" statementSuite

and generates code for the statement if no error is reported.

In [ ]:
def statement():
    if SC.sym == MALLOC:
        getSym(); 
        if SC.sym == IDENT: i = SC.val; getSym()
        else: mark("identifier expected")
        if SC.sym == COLON: getSym()
        else: mark("':' expected")
        tp = typ().val
        newDecl(i, Var(tp))
        x = find(i)
        if SC.sym == BECOMES:
            getSym(); y = expression(); call = False # x := y
            if x.tp == y.tp in {Bool, Int}: x = CG.genAssign(x, y)
            else: mark('incompatible assignment')
        elif SC.sym == LARROW:
            getSym()
            if SC.sym == IDENT: y = find(SC.val); getSym(); call = True
            else: mark('procedure identifier expected')
            if type(y) in {Proc, StdProc}:
                if len(y.res) != 1 or x.tp != y.res[0].tp: mark('incompatible call')
            else: mark('procedure expected')
        else: mark(':= or ← expected')
        return x
    elif SC.sym == IDENT: # x := y, y(...), x ← y(...)
        # type(x) == Proc, StdProc: check no result parameters needed; call, y := true, x
        # type(x) ≠ Proc, StdProc: x := designator():
        #   sym == BECOMES: assignment; call := false
        #   sym == LARROW: check result paramter match, type(y) is Proc, StdProc, 
        x = find(SC.val)
        if type(x) in {Proc, StdProc}: # call
            if x.res != []: mark('designator for result expected')
            getSym(); call, y, x = True, x, None
        elif type(x) in {Var, Ref}:
            x = designator()
            if SC.sym == BECOMES:
                getSym(); y = expression(); call = False # x := y
                if x.tp == y.tp in {Bool, Int}: x = CG.genAssign(x, y)
                else: mark('incompatible assignment')
            elif SC.sym == LARROW:
                getSym()
                if SC.sym == IDENT: y = find(SC.val); getSym(); call = True
                else: mark('procedure identifier expected')
                if type(y) in {Proc, StdProc}:
                    if len(y.res) != 1 or x.tp != y.res[0].tp: mark('incompatible call')
                else: mark('procedure expected')
            else: mark(':= or ← expected')
        else: mark("variable or procedure expected")
        if call: # call y(ap) or x ← y(ap)
            fp, ap, i = y.par, [], 0   #  list of formals, list of actuals
            if SC.sym == LPAREN: getSym()
            else: mark("'(' expected")
            if SC.sym in FIRSTEXPRESSION:
                a = expression()
                if i < len(fp):
                    if fp[i].tp == a.tp: ap.append(CG.genActualPara(a, fp[i], i))
                    else: mark('incompatible parameter')
                else: mark('extra parameter')
                i = i + 1
                while SC.sym == COMMA:
                    getSym()
                    a = expression()
                    if i < len(fp):
                        if fp[i].tp == a.tp: ap.append(CG.genActualPara(a, fp[i], i))
                        else: mark('incompatible parameter')
                    else: mark('extra parameter')
                    i = i + 1
            if SC.sym == RPAREN: getSym()
            else: mark("')' expected")
            if i < len(fp): mark('too few parameters')
            elif type(y) == StdProc:
                if y.name == 'read': x = CG.genRead(x)
                elif y.name == 'write': x = CG.genWrite(a)
                elif y.name == 'writeln': x = CG.genWriteln()
            else: x = CG.genCall(x, y, ap)
    elif SC.sym == IF:
        getSym(); x = expression();
        if x.tp == Bool: x = CG.genThen(x)
        else: mark('boolean expected')
        if SC.sym == THEN: getSym()
        else: mark("'then' expected")
        y = statementSuite()
        if SC.sym == ELSE:
            getSym()
            y = CG.genElse(x, y)
            z = statementSuite()
            x = CG.genIfElse(x, y, z)
        else:
            x = CG.genIfThen(x, y)
    elif SC.sym == WHILE:
        getSym(); t = CG.genWhile(); x = expression()
        if x.tp == Bool: x = CG.genDo(x)
        else: mark('boolean expected')
        if SC.sym == DO: getSym()
        else: mark("'do' expected")
        y = statementSuite()
        x = CG.genWhileDo(t, x, y)
    else: mark('statement expected')
    return x

Procedure `typ` parses

    type ::=
        ident |
        expression ".." expression "→" type

and returns a type descriptor if not error is reported. The array bound are checked to be constants; the lower bound must be smaller or equal to the upper bound.

In [ ]:
def typ():
    if SC.sym == IDENT:
        ident = SC.val; x = find(ident)
        if type(x) == Type: x = Type(x.val); getSym()
        else: x = None
    else: x = None
    if not x:
        x = expression()
        if SC.sym == PERIOD: getSym()
        else: mark("'.' expected")
        if SC.sym == PERIOD: getSym()
        else: mark("'.' expected")
        y = expression()
        if SC.sym == RARROW: getSym()
        else: mark("'→' expected")
        z = typ().val;
        if type(x) != Const or x.val < 0: mark('bad lower bound')
        elif type(y) != Const or y.val < x.val: mark('bad upper bound')
        else: x = Type(CG.genArray(Array(z, x.val, y.val - x.val + 1)))
    return x

Procedure `typeIds()` parses

    typedIds ::= ident ":" type {"," ident ":" type}.

and updates the top scope of symbol table; an error is reported if an identifier is already in the top scope.

In [ ]:
def typedIds():
    if SC.sym == IDENT: tid = [SC.val]; getSym()
    else: mark("identifier expected")
    while SC.sym == COMMA:
        getSym()
        if SC.sym == IDENT: tid.append(SC.val); getSym()
        else: mark('identifier expected')
    if SC.sym == COLON: getSym()
    else: mark("':' expected")
    tp = typ().val
    for i in tid: newDecl(i, Var(tp))
    while SC.sym == COMMA:
        getSym()
        if SC.sym == IDENT: tid = [SC.val]; getSym()
        else: mark("identifier expected")
        while SC.sym == COMMA:
            getSym()
            if SC.sym == IDENT: tid.append(SC.val); getSym()
            else: mark('identifier expected')
        if SC.sym == COLON: getSym()
        else: mark("':' expected")
        tp = typ().val
        for i in tid: newDecl(i, Var(tp))

Procedure `declarations(allocVar)` parses

    declarations ::= 
        {"const" ident "=" expression}
        {"type" ident "=" type}
        {"var" typedIds}
        {"procedure" ident "(" [typedIds] ")" [ "→" "(" typedIds ")" ] body}

and updates the top scope of symbol table; an error is reported if an identifier is already in the top scope. An error is also reported if the expression of a constant declarations is not constant. For each procedure, a new scope is opened for its formal parameters and local declarations, the formal parameters and added to the symbol table, and code is generated for the body. The size of the variable declarations is returned, as determined by calling paramater `allocVar`.

In [ ]:
def declarations(allocVar):
    while SC.sym == CONST:
        getSym()
        if SC.sym == IDENT: ident = SC.val; getSym()
        else: mark("constant name expected")
        if SC.sym == EQ: getSym()
        else: mark("= expected")
        x = expression()
        if type(x) == Const: newDecl(ident, x)
        else: mark('expression not constant')
    while SC.sym == TYPE:
        getSym()
        if SC.sym == IDENT: ident = SC.val; getSym()
        else: mark("type name expected")
        if SC.sym == EQ: getSym()
        else: mark("= expected")
        x = typ(); newDecl(ident, x)  #  x is of type ST.Type
    start = len(topScope())
    while SC.sym == VAR:
        getSym(); typedIds()
    varsize = allocVar(topScope(), start)
    while SC.sym == PROCEDURE:
        getSym()
        if SC.sym == IDENT: ident = SC.val; getSym()
        else: mark("procedure name expected")
        newDecl(ident, Proc([], [])) #  entered without parameters
        sc = topScope(); openScope() # new scope for parameters and body
        if SC.sym == LPAREN: getSym()
        else: mark("( expected")
        if SC.sym == IDENT: typedIds()
        fp = topScope()
        if SC.sym == RPAREN: getSym()
        else: mark(") expected")
        d = len(fp)
        if SC.sym == RARROW:
            getSym()
            if SC.sym == LPAREN: getSym()
            else: mark('( expected')
            typedIds()
            if SC.sym == RPAREN: getSym()
            else: mark(') expected')
        sc[-1].par, sc[-1].res = fp[:d], fp[d:] #  procedure parameters updated
        parsize = CG.genProcStart(ident, fp[:d], fp[d:])
        body(ident, parsize); closeScope() #  scope for parameters and body closed
    return varsize

Procedure `body` parses

    body ::= INDENT declarationBlock (statementBlock | INDENT statementBlock DEDENT) DEDENT

and returns the generated code if no error is reported.

In [ ]:
def body(ident, parsize):
    if SC.sym == INDENT: getSym()
    else: mark('indent expected')
    start = len(topScope())
    localsize = declarations(CG.genLocalVars)
    CG.genProcEntry(ident, parsize, localsize)
    if SC.sym in FIRSTSTATEMENT: x = statementBlock()
    elif SC.sym == INDENT:
        getSym(); x = statementBlock()
        if SC.sym == DEDENT: getSym()
        else: mark('dedent expected')
    else: mark('statement expected')
    CG.genProcExit(x, parsize, localsize)
    if SC.sym == DEDENT: getSym()
    else: mark('dedent expected')
    return x

Procedure `program` parses

    program ::= declarations "program" ident body
    
and returns the generated code if no error is reported. The standard identifiers are entered initially in the symbol table.

In [ ]:
def program():
    newDecl('boolean', Type(CG.genBool(Bool)))
    newDecl('integer', Type(CG.genInt(Int)))
    newDecl('true', Const(Bool, 1))
    newDecl('false', Const(Bool, 0))
    newDecl('read', StdProc([], [Var(Int)]))
    newDecl('write', StdProc([Var(Int)], []))
    newDecl('writeln', StdProc([], []))
    CG.genProgStart()
    declarations(CG.genGlobalVars)
    if SC.sym == PROGRAM: getSym()
    else: mark("'program' expected, got" + str(SC.sym))
    ident = SC.val
    if SC.sym == IDENT: getSym()
    else: mark('program name expected')
    openScope(); CG.genProgEntry(ident); x = body(ident, 0)
    closeScope(); x = CG.genProgExit(x)
    return x

Procedure `compileString(src, dstfn, target)` compiles the source as given by string `src`; if `dstfn` is provided, the code is written to a file by that name, otherwise printed on the screen. If `target` is omitted, MIPS code is generated.

In [ ]:
def compileString(src, dstfn = None, target = 'wat'):
    global CG
    if target == 'wat': import CGwat as CG
    elif target == 'mips': import CGmips as CG
    elif target == 'ast': import CGast as CG
    else: print('unknown target'); return
    try:
        SC.init(src); ST.init(); p = program()
        if dstfn == None: print(p)
        else:
            with open(dstfn, 'w') as f: f.write(p)
    except Exception as msg:
        raise Exception(str(msg))
        # print(msg)

Procedure `compileFile(srcfn, target)` compiles the file named `scrfn`, which must have the extension `.p`, and generates assembly code in a file with extension `.s`. If `target` is omitted, MIPS code is generated.

In [ ]:
def compileFile(srcfn, target = 'wat'):
    if srcfn.endswith('.p'):
        with open(srcfn, 'r') as f: src = f.read()
        dstfn = srcfn[:-2] + '.s'
        compileString(src, dstfn, target)
    else: print("'.p' file extension expected")

Sample usage (in code cell):

    cd /path/to/my/prog
    compileFile('myprog.p')